In [12]:
# impoar and path

import os
import pickle
import numpy as np
import pandas as pd

pd.set_option("display.width", 120)

PROJECT_ROOT = os.path.abspath("..")
DATA_RAW = os.path.join(PROJECT_ROOT, "data", "raw")
DATA_PROCESSED = os.path.join(PROJECT_ROOT, "data", "processed")


In [13]:
# load day 1 ouptput

prices = pd.read_csv(os.path.join(DATA_RAW, "prices.csv"), index_col=0, parse_dates=True)
returns = pd.read_csv(os.path.join(DATA_PROCESSED, "returns.csv"), index_col=0, parse_dates=True)

prices.shape, returns.shape


((1270, 25), (1269, 25))

In [14]:
# Define Rebalace date (monthly)

# Use month-end rebalancing
rebalance_dates = returns.resample("M").last().index
rebalance_dates[:5], len(rebalance_dates)


C:\Users\pawan\AppData\Local\Temp\ipykernel_35516\3865659009.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  rebalance_dates = returns.resample("M").last().index


(DatetimeIndex(['2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30', '2021-05-31'], dtype='datetime64[ns]', name='Date', freq='ME'),
 61)

In [15]:
# Rolling covarienace 

WINDOW = 60  # trading days

def rolling_covariance(returns, window, dates):
    covs = {}
    for d in dates:
        if d not in returns.index:
            continue
        end_loc = returns.index.get_loc(d)
        if end_loc < window:
            continue
        window_returns = returns.iloc[end_loc - window:end_loc]
        covs[d] = window_returns.cov()
    return covs

cov_matrices = rolling_covariance(returns, WINDOW, rebalance_dates)
len(cov_matrices)


41

In [16]:
# sanity check one covariance 

sample_date = list(cov_matrices.keys())[5]
cov_matrices[sample_date].iloc[:5, :5]


,AAPL,ADBE,AMZN,BAC,COST
AAPL,0.000224,0.000213,0.000129,0.000031,0.000093
ADBE,0.000213,0.000660,0.000185,0.000011,0.000207
AMZN,0.000129,0.000185,0.000250,0.000021,0.000063
BAC,0.000031,0.000011,0.000021,0.000234,0.000013
COST,0.000093,0.000207,0.000063,0.000013,0.000220


In [17]:
# create risk scenarios 

SCENARIOS = {
    "base": 1.0,
    "stress": 1.3,
    "calm": 0.8
}

scenario_covs = {}
for d, cov in cov_matrices.items():
    scenario_covs[d] = {k: v * cov for k, v in SCENARIOS.items()}

list(scenario_covs[sample_date].keys())


['base', 'stress', 'calm']

In [18]:
# store scenario aware covairances 

with open(os.path.join(DATA_PROCESSED, "covariance_matrices.pkl"), "wb") as f:
    pickle.dump(scenario_covs, f)

"Saved covariance_matrices.pkl"


'Saved covariance_matrices.pkl'

In [19]:
# Helper Api 

def get_covariance(date, scenario="base"):
    """
    Fetch scenario-aware covariance for a given rebalance date.
    """
    if date not in scenario_covs:
        raise KeyError("Covariance not available for this date")
    return scenario_covs[date][scenario]


In [20]:
# Quick test 

test_cov = get_covariance(sample_date, scenario="stress")
test_cov.shape


(25, 25)

In [21]:
# risk diagnostics 

# Portfolio variance under equal weights for each scenario
n_assets = returns.shape[1]
w_eq = np.ones(n_assets) / n_assets

diag = []
for s in SCENARIOS:
    var = w_eq.T @ get_covariance(sample_date, s).values @ w_eq
    vol = np.sqrt(var * 252)
    diag.append((s, vol))

diag


[('base', np.float64(0.1473847045819239)),
 ('stress', np.float64(0.168044418199806)),
 ('calm', np.float64(0.13182488731556263))]

In [22]:
# day 2 summary 

day2_summary = {
    "window_days": WINDOW,
    "scenarios": SCENARIOS,
    "num_rebalance_dates": len(cov_matrices)
}

day2_summary


{'window_days': 60,
 'scenarios': {'base': 1.0, 'stress': 1.3, 'calm': 0.8},
 'num_rebalance_dates': 41}